# Experimento: Pregunta #1

Feedback hito 2: Ya vimos que popularidad no sirve de mucho, pero seria mejor analizar que atributos propios de la cancion (tempo, loudness, ...) si sirven. Abandonar popularidad y expandir clasificacion a quizas 10 atributos. La idea seria ver como Spotify genera ese valor de "af_valence"

La pregunta habia quedado asi en el hito-2:

- ¿Es posible predecir la positividad de una canción basándose en su popularidad y tempo?

In [1]:
# %pip install pandas
# %pip install openpyxl
# %pip install seaborn
# %pip install matplotlib --user
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, FunctionTransformer, LabelEncoder, KBinsDiscretizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

import requests
import re
import string

In [ ]:
#El dataframe utilizado habia sido el siguiente:

#df_util_4 = df_spotify[["streams", "popularity", "af_tempo", "af_valence"]].copy()

In [3]:
#importamos el dataframe base:

df_spotify = pd.read_excel('../../Spotify.xlsx')

#Ajustamos columnas para contraarrestar error de formato en archivo xlsx
df_spotify['duration_ms'] = df_spotify['duration_ms']/10
df_spotify['popularity'] = df_spotify['popularity']/10
df_spotify['streams'] = df_spotify['streams']/10
df_spotify['af_danceability'] = df_spotify['af_danceability']/1000
df_spotify['af_energy'] = df_spotify['af_energy']/1000
df_spotify['af_key'] = df_spotify['af_key']/10
df_spotify['af_loudness'] = df_spotify['af_loudness']/1000
df_spotify['af_speechiness'] = df_spotify['af_speechiness']/1000
df_spotify['af_acousticness'] = df_spotify['af_acousticness']/1000
df_spotify['af_instrumentalness'] = df_spotify['af_instrumentalness']/1000
df_spotify['af_liveness'] = df_spotify['af_liveness']/1000
df_spotify['af_valence'] = df_spotify['af_valence']/1000
df_spotify['af_tempo'] = df_spotify['af_tempo']/1000
df_spotify['af_time_signature'] = df_spotify['af_time_signature']/10

Atributos propios de la cancion a conservar:

- duration_ms
- explicit
- af_danceability
- af_energy
- af_key
- af_loudness
- af_mode
- af_speechiness
- af_acousticness
- af_instrumentalness
- af_liveness
- af_valence
- af_tempo
- af_time_signature

In [ ]:
#Partimos seleccionando todo atributo propio a la cancion

df_test_1 = df_spotify[["duration_ms", "explicit", "af_danceability", "af_energy", "af_key", "af_loudness", "af_mode", "af_speechiness", "af_acousticness", "af_instrumentalness", "af_liveness", "af_valence", "af_tempo", "af_time_signature"]].copy()

discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')

X_1 = df_test_1.drop(columns = 'af_valence')
y_reshaped = df_test_1["af_valence"].values.reshape(-1,1) #Redimensionamos datos a matriz de una sola columna

#aplanamos datos transformados por discretizador y creamos serie de pandas.
#Las etiquetas discretizadas representan positividad en una escala del 0 al 9, siendo 0 poco positivo, y 9 mucho.
y = pd.Series(discretizer.fit_transform(y_reshaped).flatten(), name='af_valence', dtype=int) 

#Escalamos y normalizamos los datos del dataframe
X_scaled_1 = pd.DataFrame(StandardScaler().fit_transform(X_1), columns = X_1.columns)

In [5]:
X_scaled_1

,duration_ms,explicit,af_danceability,af_energy,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_tempo,af_time_signature
0,0.223272,-0.708129,-2.345716,-0.740317,1.487257,-1.676977,-1.086527,-0.472064,-1.226130,-0.371915,-0.729222,0.027485,0.072313
1,-1.624309,1.412173,-0.088171,0.483934,-1.217273,-0.031738,0.920363,-0.238680,-0.374582,-0.371916,-0.363745,1.497668,0.072313
2,0.178925,-0.708129,-0.108959,1.213180,0.946351,0.932100,0.920363,0.568275,-0.478617,-0.371916,-0.363745,1.487958,0.072313
3,-0.697248,1.412173,-2.270880,0.793311,1.487257,1.805701,0.920363,1.161624,-1.310900,-0.371876,-0.070733,-0.389899,0.072313
4,0.711734,-0.708129,0.676817,-0.161339,0.134992,-0.890772,-1.086527,-0.824117,2.087586,-0.371916,-0.785934,1.251108,0.072313
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009045,0.064672,-0.708129,0.797385,1.080590,1.487257,1.023404,-1.086527,0.054039,-0.632744,-0.371916,1.192684,0.606631,0.072313
1009046,1.434313,-0.708129,-2.416394,-0.899425,0.946351,-1.274105,0.920363,-0.448330,1.740801,-0.371916,-0.697716,0.433699,0.072313
1009047,0.640407,-0.708129,-0.408302,-0.351385,0.946351,-1.777162,-1.086527,0.057995,0.211098,-0.371890,-0.250321,-0.507165,0.072313
1009048,0.469563,-0.708129,0.427364,1.085010,-0.946820,0.751625,-1.086527,-1.373954,1.328514,-0.371916,-1.113604,-0.414047,0.072313


In [7]:
y

0          2
1          3
2          8
3          4
4          0
          ..
1009045    8
1009046    0
1009047    2
1009048    6
1009049    3
Name: af_valence, Length: 1009050, dtype: int64

## Reduccion de atributos

### Sin reducir atributos:

In [8]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_scaled_1, y, test_size=0.3, random_state=0, stratify=y)

In [64]:
#KNeighborsClassifier: GridSearch

tuned_parameters = {'n_neighbors': list(range(1, 16, 1)), 'weights': ['uniform', 'distance']}

score = 'f1_macro'

cls = KNeighborsClassifier(n_jobs = -1)

clf = GridSearchCV(cls, param_grid = tuned_parameters, scoring = score, cv = 10)

clf.fit(X_train_1, y_train_1)

print("Mejor combinación de parámetros:")
print(clf.best_params_)

Mejor combinación de parámetros:
{'n_neighbors': 5, 'weights': 'distance'}


In [10]:
kn_clf_1 = KNeighborsClassifier(n_neighbors=5, weights = 'distance', n_jobs = -1)

kn_clf_1.fit(X_train_1, y_train_1)

y_pred_1 = kn_clf_1.predict(X_test_1)

print(classification_report(y_test_1, y_pred_1))

#Cross-Validation:
scoring = ['precision_macro', 'recall_macro', 'accuracy', 'f1_macro']

cv_results = cross_validate(kn_clf_1, X_scaled_1, y, cv=10, scoring=scoring)
pd.DataFrame.from_dict(cv_results)

print("Resultados de 10-fold Cross-Validation:")
print('')
print("precision_macro promedio: ", cv_results['test_precision_macro'].mean().round(3))
print("recall_macro promedio: ", cv_results['test_recall_macro'].mean().round(3))
print("accuracy promedio: ", cv_results['test_accuracy'].mean().round(3))
print("f1_macro promedio: ", cv_results['test_f1_macro'].mean().round(3))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28661
           1       1.00      1.00      1.00     11122
           2       1.00      1.00      1.00     21108
           3       1.00      1.00      1.00     38446
           4       1.00      1.00      1.00     35541
           5       1.00      1.00      1.00     49062
           6       1.00      1.00      1.00     42497
           7       1.00      1.00      1.00     35811
           8       1.00      1.00      1.00     28952
           9       1.00      1.00      1.00     11515

    accuracy                           1.00    302715
   macro avg       1.00      1.00      1.00    302715
weighted avg       1.00      1.00      1.00    302715

Resultados de 10-fold Cross-Validation:

precision_macro promedio:  0.989
recall_macro promedio:  0.988
accuracy promedio:  0.99
f1_macro promedio:  0.988


### Reduccion a 10 atributos:

#### K-best: 
- mutual_info_classifier

In [14]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2, f_classif

selector = SelectKBest(mutual_info_classif, k=10)

X_train_selected_1 = pd.DataFrame(selector.fit_transform(X_train_1, y_train_1), columns=X_scaled_1.columns[selector.get_support()])
X_test_selected_1 = pd.DataFrame(selector.transform(X_test_1), columns=X_scaled_1.columns[selector.get_support()])

In [15]:
X_train_selected_1

,duration_ms,af_danceability,af_energy,af_key,af_loudness,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_tempo
0,0.592243,0.726707,0.121520,-0.676367,-1.162551,-1.239462,2.253271,-0.371916,-0.779633,-0.483202
1,-0.094020,-1.144186,-0.678441,-0.135461,-0.604783,-0.270325,0.858428,-0.371916,-0.140048,-0.272540
2,0.178925,-0.108959,1.213180,0.946351,0.932100,0.568275,-0.478617,-0.371916,-0.363745,1.487958
3,-0.935485,0.851434,0.112681,-1.487726,-0.448465,-1.128703,-1.384110,-0.241797,-0.540182,-0.297268
4,-0.414944,0.718392,-0.072945,0.405445,0.881652,-1.172215,-0.910171,-0.371913,-0.688264,1.350089
...,...,...,...,...,...,...,...,...,...,...
706330,-1.076741,1.134146,0.475094,-1.487726,0.869218,1.038998,2.102998,-0.371916,-0.316485,-0.250871
706331,0.825033,0.144651,0.620944,0.405445,0.080882,-0.218901,-0.293666,-0.371911,1.164328,-0.320211
706332,0.240405,0.701762,0.002189,1.487257,1.819201,1.628392,-0.108714,1.827712,-0.612648,-0.320837
706333,0.307309,0.261063,0.810990,-1.487726,0.746651,-0.025074,-0.652010,1.139941,1.564463,0.143778


In [17]:
y_train_1

868290    3
734102    0
694479    8
711905    3
100664    7
         ..
667307    9
341316    7
892778    2
261551    5
790300    0
Name: af_valence, Length: 706335, dtype: int64

In [18]:
#KNeighborsClassifier: GridSearch

tuned_parameters = {'n_neighbors': list(range(1, 16, 1)), 'weights': ['uniform', 'distance']}

score = 'f1_macro'

cls = KNeighborsClassifier(n_jobs = -1)

clf = GridSearchCV(cls, param_grid = tuned_parameters, scoring = score, cv = 10)

clf.fit(X_train_selected_1, y_train_1)

print("Mejor combinación de parámetros:")
print(clf.best_params_)

Mejor combinación de parámetros:
{'n_neighbors': 13, 'weights': 'distance'}


In [20]:
kn_clf_2 = KNeighborsClassifier(n_neighbors=13, weights = 'distance', n_jobs = -1)

kn_clf_2.fit(X_train_selected_1, y_train_1)

y_pred_1 = kn_clf_2.predict(X_test_selected_1)

print(classification_report(y_test_1, y_pred_1))

#Cross-Validation:
scoring = ['precision_macro', 'recall_macro', 'accuracy', 'f1_macro']

X_scaled_selected_1 = pd.DataFrame(selector.transform(X_scaled_1), columns=X_scaled_1.columns[selector.get_support()])

cv_results = cross_validate(kn_clf_2, X_scaled_selected_1, y, cv=10, scoring=scoring)
pd.DataFrame.from_dict(cv_results)

print("Resultados de 10-fold Cross-Validation:")
print('')
print("precision_macro promedio: ", cv_results['test_precision_macro'].mean().round(3))
print("recall_macro promedio: ", cv_results['test_recall_macro'].mean().round(3))
print("accuracy promedio: ", cv_results['test_accuracy'].mean().round(3))
print("f1_macro promedio: ", cv_results['test_f1_macro'].mean().round(3))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28661
           1       1.00      1.00      1.00     11122
           2       1.00      1.00      1.00     21108
           3       1.00      1.00      1.00     38446
           4       1.00      1.00      1.00     35541
           5       1.00      1.00      1.00     49062
           6       1.00      1.00      1.00     42497
           7       1.00      1.00      1.00     35811
           8       1.00      1.00      1.00     28952
           9       1.00      1.00      1.00     11515

    accuracy                           1.00    302715
   macro avg       1.00      1.00      1.00    302715
weighted avg       1.00      1.00      1.00    302715

Resultados de 10-fold Cross-Validation:

precision_macro promedio:  0.989
recall_macro promedio:  0.988
accuracy promedio:  0.99
f1_macro promedio:  0.989


#### En base a un modelo:

- SVC